In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sms-spam-collection-dataset/spam.csv


In [2]:
data=pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv")

In [3]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
data.shape

(5572, 5)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [6]:
X=data["v2"]
Y=data["v1"].apply(lambda x : 1 if x =="spam" else 0)

In [7]:
Y

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: v1, Length: 5572, dtype: int64

In [8]:
X

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will �_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: v2, Length: 5572, dtype: object

In [9]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords


In [10]:
ps = PorterStemmer()

In [11]:
def clean_text(X):
    newlist=[]
    for i in range(len(X)):
        removepun=re.sub(r'[^\w\s]', '',X[i].lower())
        tokens=word_tokenize(removepun)
        clean=[]
        for token in tokens:
            if token not in stopwords.words('english'):
                if token.isalpha():
                    stem=ps.stem(token)
                    clean.append(stem)
        newlist.append(" ".join(clean))
    return newlist
            
        
    
    


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [13]:
cv=CountVectorizer(min_df=5)
XClean=clean_text(X)

In [14]:
xtrain,xtest,ytrain,ytest=train_test_split(XClean,Y,test_size=0.2)

In [15]:
XCV=cv.fit_transform(xtrain)

In [16]:
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression


In [17]:
estimators={"LinearSVC":LinearSVC(),
           "DecisionTreeClassifier":DecisionTreeClassifier(),
           "BernoulliNB":BernoulliNB(),
           "LogisticRegression":LogisticRegression()}

In [18]:
for model in estimators.values():
    model.fit(XCV,ytrain)
    print(f"{model} :{model.score(cv.transform(xtest),ytest)}")

LinearSVC() :0.9757847533632287
DecisionTreeClassifier() :0.9614349775784753
BernoulliNB() :0.979372197309417
LogisticRegression() :0.9748878923766816


In [19]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,accuracy_score


In [20]:
model=BernoulliNB()
model.fit(XCV,ytrain)
yp=model.predict(cv.transform(xtest))

In [21]:
print(classification_report(ytest,yp))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       961
           1       0.99      0.86      0.92       154

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [22]:
print(confusion_matrix(ytest,yp))

[[960   1]
 [ 22 132]]


In [23]:
model1=LinearSVC()
model1.fit(XCV,ytrain)
yp1=model1.predict(cv.transform(xtest))

In [24]:
print(classification_report(ytest,yp1))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       961
           1       0.96      0.86      0.91       154

    accuracy                           0.98      1115
   macro avg       0.97      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [25]:
print(confusion_matrix(ytest,yp1))

[[955   6]
 [ 21 133]]


In [26]:
def predict(text,model):
    xc=clean_text(text)
    value=model.predict(cv.transform(xc))
    if value==0:
        print("NOT SPAM")
    else:
        print("SPAM")

In [57]:
text=["Weekend Offer: Get Flipkart voucher worth Rs. 500"]
text1=["Less than 4 days left! Claim your Flipkart voucher worth Rs. 500"]
#spam mail 

In [58]:
predict(text,model)

SPAM


In [59]:
predict(text1,model)

SPAM
